In [29]:
# Functions
import json
import os
from lxml import etree
import random
from multiprocessing import Pool
import sys

In [30]:
def make_filelist(path):
    with open(path) as f:
        return [x for x in f.read().split('\n') if x]

In [31]:
def get_people(file):
    tree = etree.parse(file)
    people = [x for x in tree.xpath("//NER") if x.text == 'PERSON']
    tokens = [x.getparent() for x in people]
    l = []
    prev_end = 0
    name_id = 0
    
    for tok in tokens:
        d = {}
        data = [x.text for x in tok.getchildren()]
        # TODO this is dumb surely there's a better way
        d['word']=data[0]
#        d['lemma']=data[1]
        d['start']=data[2]
        curr_start = data[2]
        d['end']=data[3]
        
        # check if there is something in the XML that already provides for this
        if (int(curr_start) - int(prev_end)) == 1:
            d['adjacent'] = True
            d['name_id'] = name_id
            # go back to the previous dictionary in the list and assert True
            if len(l) > 0: # skip zeroth case
                l[-1]['adjacent'] = True
                l[-1]['name_id'] = name_id
        else:
            d['adjacent'] = False
            d['name_id'] = None
        
        if len(l) > 0 and l[-1]['adjacent'] == True and d['adjacent'] == False:
            name_id += 1
        
        d['pos']=data[4]
        d['ner']=data[5]
        d['speaker']=data[6]
        d['file'] = tok.base
        prev_end = data[3]
        l.append(d)
    
    return l

In [32]:
def pool_people(filelist,path):
    with Pool() as p:
        l = p.map(get_people,filelist)
    data = [item for sublist in l for item in sublist] # flatten list
    filename = os.path.split(path)[-1].split('.txt')[0] + '.json'
    out = '/scratch/groups/malgeehe/celebs/chicago_people/json/'
    fp = os.path.join(out,filename)
    with open(fp,'w') as f:
        f.write(json.dumps(data,indent=2))
        f.close()
    return fp

In [33]:
def get_people_xml(path):
    '''Converts list of paths to xml files into json of people data'''
    print('get_people_xml operating on {}'.format(path))
    files = make_filelist(path)
    output = pool_people(files,path)
    print('written to {}'.format(output))
    return

In [35]:
path = [x for x in sys.argv if '/scratch/groups/malgeehe/celebs/chicago_people/filelists/group_' in x]
if len(path) == 1:
    path = str(path[0])
    get_people_xml(path)
else:
    print('problem with sys.argv: {}'.format(sys.argv))

problem with sys.argv: ['/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/tmp/jupyter/kernel-d967cd7b-5067-40c0-a050-586a0e5646c3.json']
